# Lab 4: Tools - Dando Superpoderes ao seu Agente

## 🎯 O que você vai aprender neste Lab

**Tools** permitem que agentes "ajam" no mundo real. São o componente de "Acting" no ciclo ReAct (Reasoning + Acting).

### Por que Tools são importantes?

1. **Ação no mundo real**: Consultar APIs, bancos de dados, sistemas externos
2. **Capacidades expandidas**: Cálculos, buscas, operações que LLMs não fazem bem sozinhos
3. **Integração**: Conectar o agente a qualquer sistema via código Python
4. **Controle**: Você define exatamente o que o agente pode fazer

### Como o LLM decide usar uma Tool?

O LLM usa duas informações para decidir:
1. **Descrição da tool**: Quando usar (inferida da docstring)
2. **Parâmetros**: Como chamar (inferidos da assinatura da função)

```
┌─────────────────────────────────────────────────────┐
│            ANATOMIA DE UMA TOOL                      │
│                                                      │
│   @tool                                             │
│   def minha_tool(param: tipo) -> tipo:              │
│       \"\"\"Descrição: QUANDO usar a tool\"\"\"         │
│       # Código: O QUE a tool faz                    │
│       return resultado                              │
│                                                      │
│   Nome: minha_tool (ou customizado)                 │
│   Descrição: Da docstring                           │
│   Parâmetros: Da assinatura (param: tipo)           │
└─────────────────────────────────────────────────────┘
```

## ⚙️ Setup - Configuração do Ambiente

Instalação das bibliotecas e configuração do MLflow.

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

## 🧮 Exemplo: Calculadora Básica

Vamos criar uma calculadora como primeira tool. Observe como a **docstring** e os **tipos** são usados pelo LLM para entender quando e como usar a ferramenta.

### A importância da descrição

A docstring é passada ao LLM como descrição da tool. Ela deve explicar:
- **O que** a tool faz
- **Quando** usar (em quais situações)
- **Limitações** (se houver)

In [0]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

### Criando o agente com a calculadora

In [ ]:
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

### Testando com números decimais

A tool é invocada porque a descrição menciona "real numbers" e os inputs são decimais.

In [0]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

## ⚠️ Problema: Descrição Genérica Demais

A descrição pode ter grande impacto no comportamento! Com uma descrição genérica, o LLM pode:
- **Não usar a tool** quando deveria
- **Fazer o cálculo sozinho** (possivelmente com erros)

### Exemplo: Inteiros vs Decimais

Com a descrição "real numbers", o LLM pode decidir que inteiros (3 * 4) não precisam da calculadora:

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers.  (results vary from run to run)

### Mesmo com decimais explícitos (3.0 * 4.0)

O comportamento pode variar entre execuções - às vezes usa a tool, às vezes não:

This often does not invoke the tool though the input are real numbers. (results vary from run to run)

## ✅ Solução: Descrições Detalhadas

O LangChain suporta descrições aprimoradas através de:

1. **`parse_docstring=True`**: Extrai descrições dos argumentos no estilo Google
2. **`description=`**: Sobrescreve a descrição padrão
3. **Nome customizado**: Renomeia a tool

### Melhorando a calculadora

Vamos adicionar uma descrição explícita que inclui "even if they are integers":

## Adding a more detailed description
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. You can rename the tool and change its description. This can be effective when you are sharing a standard tool but would like agent-specific instructions.

In [0]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

### Verificando via MLflow/LangSmith

Você pode verificar os metadados da tool no trace:
- **Nome**: "calculator" (customizado)
- **Descrição**: Inclui "even if they are integers"
- **Parâmetros**: Com descrições detalhadas de cada argumento

Let's check our [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r) to see the tool description.

### Agora funciona com inteiros também!

## 🎮 Sua Vez - Crie sua Própria Tool!

Crie uma tool customizada. Algumas ideias:
- Conversor de temperatura (Celsius ↔ Fahrenheit)
- Calculadora de IMC
- Gerador de números aleatórios
- Formatador de datas

### Template:

## Try your own.
Create a tool of your own and try it here!

In [0]:

@tool
def your_tool(
    a: float, b: float, 
) -> float:
    """Perform your favorite operation

    Args:
        a (float): operator a description
        b (float): operator b description

    Returns:
        float: description
    """
    pass

## 📚 Resumo e Próximos Passos

### O que aprendemos:

| Elemento | Propósito | Como definir |
|----------|-----------|--------------|
| **Nome** | Identificação da tool | Nome da função ou `@tool("nome")` |
| **Descrição** | QUANDO usar | Docstring ou `description=` |
| **Parâmetros** | COMO chamar | Tipos na assinatura + `parse_docstring=True` |

### Boas práticas para descrições:
1. Seja **específico** sobre quando usar
2. Inclua **casos de borda** (inteiros, vazios, etc.)
3. Use **parse_docstring=True** para documentar parâmetros
4. **Teste** com diferentes inputs para verificar o comportamento

### Próximo Lab:
No **Lab 5 - MCP (Model Context Protocol)**, você aprenderá a conectar seu agente a servidores de ferramentas externos usando um padrão aberto!

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)